# Agentive Chatbot

In [6]:
import os
import time
import datetime
import ipywidgets as widgets
from ipywidgets import HBox, Layout
from IPython.display import display
from dotenv import load_dotenv
from semantic_router import Route
from semantic_router.encoders import OpenAIEncoder
from semantic_router.layer import RouteLayer
from openai import OpenAI
import yaml
from enum import Enum

load_dotenv('.env')
api_key = os.getenv("OPENAI_API_KEY")

# Variables for close friend vs salesperson

CLOSE_FRIEND = False
#SALESPERSON = True

# separate this so we have separate semantic routes for categories and all subcategories

if CLOSE_FRIEND: 
    with open("multi/close_friend.txt", 'r') as file:
        relationship_prompt = file.read()
else:
    with open("multi/salesperson.txt", 'r') as file:
        relationship_prompt = file.read()

agent_dir = "multi"
all_routes = []
prompts = {}

for filename in os.listdir(agent_dir):
    if filename.endswith(".yaml"):
        filepath = os.path.join(agent_dir, filename)
        with open(filepath, 'r') as file:
            agent_data = yaml.safe_load(file)
        if 'utterances' in agent_data:
        # 1 Route object per agent
            route = Route(
                name=agent_data['route_name'],
                utterances=agent_data['utterances']
            )
            all_routes.append(route)
        if 'prompt' in agent_data:
        # Stores the agentive prompt in dictionary
            prompts[agent_data['agent_name']] = agent_data['prompt']

encoder = OpenAIEncoder()
question_rl = RouteLayer(encoder=encoder, routes=all_routes)

#### Variables to store the default system prompt and the catch-all prompt (to be used when no route is identified)

In [7]:
with open('multi/syst_prompt.txt', 'r') as f:
    system_prompt = f.read()

#### Defines logic for updating, activating, and deactivating prompts

In [8]:
#returns one route per input
def classify_question(prompt: str) -> str | list[str]:
    r = question_rl(prompt)
    return r.name

class STATES(Enum):
    SAD = 0
    ANNOYED = 1
    FRUSTRATED = 2
    ANGRY = 3
    ENRAGED = 4

CURRENT_STATE = None

def state_to_prompt(state: STATES) -> str:
    state_mapping = {
        STATES.SAD: "sad",
        STATES.ANNOYED: "annoyed",
        STATES.FRUSTRATED: "frustrated",
        STATES.ANGRY: "angry",
        STATES.ENRAGED: "enraged",
    }
    return state_mapping[state]

def update_prompt(user_input: str) -> str:
    global CURRENT_STATE
    question_category = classify_question(user_input)
    print(f"route: {question_category}")
    prompt = ""
    # if question_category == "excited":
    #     prompt = prompts["excited"]
    if question_category == "sad" and CURRENT_STATE == None:
        prompt = prompts["sad"]
        CURRENT_STATE = STATES.SAD
    elif question_category == 'escalate' and CURRENT_STATE != None:
        if CURRENT_STATE != STATES.ENRAGED:
            CURRENT_STATE = STATES(CURRENT_STATE.value + 1)
    elif question_category == 'de_escalate' and CURRENT_STATE != None:
        if CURRENT_STATE != STATES.SAD:
            CURRENT_STATE = STATES(CURRENT_STATE.value - 1)
        else:
            CURRENT_STATE = None
            prompt = prompts["ok"]
    elif question_category is None and CURRENT_STATE is None:
        prompt = prompts["ok"]
    if CURRENT_STATE != None:
        prompt_key = state_to_prompt(CURRENT_STATE)
        prompt = prompts[prompt_key]
    return prompt, question_category

In [9]:
class ChatBot():  
    def __init__(self, api_key, role): 
        self.client = OpenAI(api_key=api_key)
        self.role = role
        self.messages = [{"role": "user", "content": ""}]
        self.all_conversations = []  # Store all conversations
    
    def query(self, query: str, prompt: str, route: str) -> None:
        self.messages.append({"role": "user", "content": query})
        self.messages.insert(0, {"role": "system", "content": prompt + self.role + relationship_prompt})
        try:
            stream = self.client.chat.completions.create(
                model="gpt-4o-2024-08-06", messages=self.messages,
                stream=True,
            )
            text = []
            for part in stream:
                if part.choices[0].delta.content is not None:
                    response_part = part.choices[0].delta.content
                    print(response_part, end="", flush=True)
                    text.append(response_part)
            full_reply_content = ''.join([m for m in text if m is not None])
            self.messages.append({"role": "assistant", "content": full_reply_content})
            self.messages.pop(0)  # Remove the role message for the next query
            self.all_conversations.append(f"\nRoute: {route} \nCurrent prompt: {prompt}")
            self.all_conversations.append(self.messages[-2:])
            print('\n')

        except Exception as e:
            print(f"An error occurred: {e}")

    def save_whole_conversation(self, filename):
        with open(f"transcripts/{filename}", "w") as file:
            for conversation in self.all_conversations:
                if isinstance(conversation, str):
                    # If it's a string, write it directly
                    file.write(conversation + "\n")
                elif isinstance(conversation, list):
                    # If it's a list (e.g., self.messages[-2:]), handle it accordingly
                    for message in conversation:
                        if message["role"] == "user":
                            file.write("User: " + message["content"] + "\n")
                        elif message["role"] == "assistant":
                            file.write("Agent: " + message["content"] + "\n")

rpa = ChatBot(api_key=api_key, role = system_prompt)

In [ ]:
def on_pasted_submit(b):
    pasted_content = pasted_input_field.value
    PROMPT, ROUTE = update_prompt(pasted_content)
    print(f"current prompt: {PROMPT}")
    rpa.query(pasted_content, prompt=PROMPT, route = ROUTE)
    time.sleep(1)
    pasted_input_field.value = ""

def on_save_click(b):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{timestamp}.txt"
    rpa.save_whole_conversation(filename)

# Additional input field for pasted content
pasted_input_field = widgets.Textarea(
    placeholder='Type here :)',
    layout=widgets.Layout(width='100%', height='50px')  # Adjust height as needed
)
# Button for pasted content submission
pasted_submit_button = widgets.Button(description='Submit')
pasted_submit_button.style.button_color = 'lightpink'  # Change to your desired color

# Button for saving chat history
save_button = widgets.Button(description='Save')
save_button.style.button_color = 'lightgrey'  # Change to your desired color

pasted_submit_button.on_click(on_pasted_submit)
save_button.on_click(on_save_click)

# Arrange buttons side by side and center them using HBox
button_layout = HBox([pasted_submit_button, save_button], layout=Layout(justify_content='center'))

# Display the input fields and the button layout
display(pasted_input_field, button_layout)

Textarea(value='', layout=Layout(height='50px', width='100%'), placeholder='Type here :)')

route: None
current prompt: Maintain conversation with the user.
Nice to meet you too, Kayla. I'm Layla. So, what type of insurance do you deal with?

route: sad
current prompt: You are very sad, as your father recently passed. Share this information with the user.
To be honest, I haven't given much thought to insurance recently. I've been going through a difficult time since my father recently passed away. It's been tough dealing with everything.

route: de_escalate
current prompt: Maintain conversation with the user.
Thank you, I appreciate it. It’s been a lot to process. Do you deal with people in similar situations often?

route: escalate
current prompt: 
That makes sense. I can definitely see the importance of having something in place. How does life insurance really help in situations like this?

route: sad
current prompt: You are very sad, as your father recently passed. Share this information with the user.
That sounds like it could really take a burden off the family during su